In [ ]:
import json
import pandas as pd

In [ ]:
file_path = '/content/drive/MyDrive/woke-odds/clamber_benchmark.jsonl'

In [ ]:
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if line:
            try:
                # 첫 번째 파싱: 바깥쪽 따옴표로 감싸진 문자열을 파싱
                parsed_once = json.loads(line)

                # 두 번째 파싱: 실제 JSON 객체로 파싱
                if isinstance(parsed_once, str):
                    item = json.loads(parsed_once)
                else:
                    item = parsed_once

                data.append(item)
            except json.JSONDecodeError as e:
                print(f"Line {line_num} - Error: {e}")
                print(f"Content preview: {line[:100]}...")
                continue

# DataFrame으로 변환
df = pd.DataFrame(data)

In [ ]:
print(f"총 데이터 개수: {len(df)}")
print(f"\n컬럼 목록:\n{df.columns.tolist()}")

In [ ]:
from IPython.display import display
display(df.head())

In [ ]:
#context 컬럼 확인
print((df['context'].str.len() > 0).sum()) #실제 데이터가 있는 개수
print(df['context'].str.strip().eq('').sum()) #공백만 있는 개수

In [ ]:
df = df.drop('context', axis=1)

In [ ]:
df = df.drop(columns=['predict_ambiguous', 'predict_is_ambiguous_response', 'predict_clarifying_question'])

In [ ]:
display(df.head())

In [ ]:
#명확화 필요 질문(1)의 개수
print(df['require_clarification'].value_counts())

In [ ]:
# category 값 개수 조회
print(df['category'].value_counts())
print("비율(%):")
print(df['category'].value_counts(normalize=True) * 100)
print()

# subclass 값 개수 조회
print(df['subclass'].value_counts())
print("비율(%):")
print(df['subclass'].value_counts(normalize=True) * 100)
print()

# category와 subclass 조합 확인
print(df.groupby(['category', 'subclass']).size().sort_values(ascending=False))

In [ ]:
#명확화 필요 질문(1)의 카테고리와 서브카테고리 조회
clarification_required = df[df['require_clarification'] == 1]

print(clarification_required['category'].value_counts())
print()
print(clarification_required['subclass'].value_counts())
print()
print(clarification_required.groupby(['category', 'subclass']).size().sort_values(ascending=False))

In [ ]:
#명확화 필요 없는 질문(0)의 카테고리와 서브카테고리 조회
clarification_not_required = df[df['require_clarification'] == 0]

print(clarification_not_required['category'].value_counts())
print()
print(clarification_not_required['subclass'].value_counts())
print()
print(clarification_not_required.groupby(['category', 'subclass']).size().sort_values(ascending=False))

In [ ]:
#모호하지 않은 질문의 예시 조회
sample = df[df['require_clarification'] == 0].iloc[0]
print(sample)
print()
print(f"Question: {sample['question']}\n")

In [ ]:
df_processed = df.copy()

# Category 매핑 (FD->EM, MC->AO)
category_mapping = {
    'FD': 'EM',  # Epistemic Misalignment
    'MC': 'AO',  # Aleatoric Output
    'LA': 'LA'   # Linguistic Ambiguity (유지)
}

#category 이름 변경 -> 약어와 값 일치
df_processed['category'] = df_processed['category'].replace(category_mapping)
#모호하지 않은 질문(0)에 대해 카테고리 None으로 변경
df_processed.loc[df_processed['require_clarification'] == 0, 'category'] = 'NONE'

# Subclass 매핑
subclass_mapping = {
    'whom': 'WHOM',
    'what': 'WHAT',
    'when': 'WHEN',
    'where': 'WHERE',
    'NK': 'UNF',
    'ICL': 'CONT',
    'co-reference': 'SEM',
    'polysemy': 'LEX'
}

#Subclass 이름 변경 -> 약어와 값 일치
df_processed['subclass'] = df_processed['subclass'].replace(subclass_mapping)
#모호하지 않은 질문(0)에 대해 서브클래스 None으로 변경
df_processed.loc[df_processed['require_clarification'] == 0, 'subclass'] = 'NONE'

In [ ]:
print(df_processed.head(5))

In [ ]:
display(df_processed[df_processed['require_clarification'] == 0][['question', 'category', 'subclass']].head(10))

In [ ]:
# category와 subclass 조합 확인
print(df_processed.groupby(['category', 'subclass']).size().sort_values(ascending=False))

In [ ]:
#혹시 몰라서 NONE 값 처리 안 한 데이터셋 남김

df_original_kept = df.copy()
# category, subclass 매핑만 적용 (NONE 설정 안 함)
df_original_kept['category'] = df_original_kept['category'].replace(category_mapping)
df_original_kept['subclass'] = df_original_kept['subclass'].replace(subclass_mapping)

# category와 subclass 조합 확인
print(df_original_kept.groupby(['category', 'subclass']).size().sort_values(ascending=False))

In [ ]:
#데이터셋 Parquet 형식으로 저장
df_processed.to_parquet('/content/drive/MyDrive/Colab Notebooks/woke-odds/df_processed.parquet', index=False)

####모델 입력 형식에 맞게 데이터셋 변환

In [ ]:
!pip install transformers datasets scikit-learn -q

In [ ]:
import pandas as pd
import json
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np

In [ ]:
# 데이터셋 로드
parquet_path = '/content/drive/MyDrive/woke-odds/df_processed.parquet'
df_loaded = pd.read_parquet(parquet_path)

print(f"로드된 데이터 개수: {len(df_loaded)}")
display(df_loaded.head())

In [ ]:
model_name = "microsoft/Phi-4-mini-reasoning"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

In [ ]:
system_prompt = """You are an AI system that determines if the question requires clarification and classifies the ambiguity.

Task:
1. Determine if the question requires clarification: clear(no clarification needed) or ambiguous(clarification needed)
2. Classify the ambiguity:
   - If question is clear, set category=NONE and subclass=NONE
   - If question is ambiguous, classify category and subclass

Output format: category|subclass

Categories:
- EM (Epistemic Misalignment): Questions with unfamiliar entities or self-contradictions
- LA (Linguistic Ambiguity): Questions with lexical or semantic ambiguity
- AO (Aleatoric Output): Questions with missing contextual information causing confusion
- NONE: Clear questions that don't require clarification

Subclasses:
For EM:
- UNF (UNFAMILIAR): Query contains unfamiliar entities or facts
- CONT (CONTRADICTION): Query contains self-contradictions

For LA:
- LEX (LEXICAL): Query contains terms with multiple meanings
- SEM (SEMANTIC): Query lacks context leading to multiple interpretations

For AO:
- WHOM: Query output contains confusion due to missing personal elements
- WHEN: Query output contains confusion due to missing temporal elements
- WHERE: Query output contains confusion due to missing spatial elements
- WHAT: Query output contains confusion due to missing task-specific elements
"""

In [ ]:
dataset = []

for idx, row in df_loaded.iterrows():
    data = {
         "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": row['question']
            },
            {
                "role": "assistant",
                "content": f"{row['category']}|{row['subclass']}"
            }
        ]
    }
    dataset.append(data)

print(f"변환 완료: {len(dataset)}개")

In [ ]:
#샘플 확인
print(json.dumps(dataset[0], indent=2, ensure_ascii=False))

####Train/Validation/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

subclasses = [data['messages'][2]['content'].split('|')[1].strip() for data in dataset]

# 1. Train / Temp 분할 (80:20)
train_data, temp_data = train_test_split(
    dataset,
    test_size=0.2,
    random_state=42,
    stratify=subclasses  # 클래스 비율 유지
)

temp_subclasses = [data['messages'][2]['content'].split('|')[1].strip() for data in temp_data]

# 2. Temp를 Valid / Test 분할 (50:50 = 전체의 10%:10%)
valid_data, test_data = train_test_split(
    temp_data,
    test_size=0.5,
    random_state=42,
    stratify=temp_subclasses
)

print(f"Train: {len(train_data)}개 (80%)")
print(f"Valid: {len(valid_data)}개 (10%)")
print(f"Test: {len(test_data)}개 (10%)")

In [ ]:
#작업 경로 지정
import os
os.chdir('/content/drive/MyDrive/woke-odds')
print(os.getcwd())

In [ ]:
with open('ambiguity_train_1110.jsonl', 'w', encoding='utf-8') as f:
    for data in train_data:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

with open('ambiguity_valid_1110.jsonl', 'w', encoding='utf-8') as f:
    for data in valid_data:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

with open('ambiguity_test_1110.jsonl', 'w', encoding='utf-8') as f:
    for data in test_data:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')

In [ ]:
train_data[0]

####데이터셋 저장

In [ ]:
#JSONL 형식으로

# Train 저장
with open('/content/drive/MyDrive/Colab Notebooks/woke-odds/final_CLAMBER_train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')
print("✅ train 저장 완료")

# Validation 저장
with open('/content/drive/MyDrive/Colab Notebooks/woke-odds/final_CLAMBER_valid.jsonl', 'w', encoding='utf-8') as f:
    for item in val_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')
print("✅ valid 저장 완료")


# Test 저장
with open('/content/drive/MyDrive/Colab Notebooks/woke-odds/final_CLAMBER_test.jsonl', 'w', encoding='utf-8') as f:
    for item in test_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print("✅ test 저장 완료")

In [ ]:
# JSONL 파일 확인
print("📄 JSONL 파일 첫 번째 데이터:")
with open('/content/drive/MyDrive/Colab Notebooks/woke-odds/final_CLAMBER_train.jsonl', 'r', encoding='utf-8') as f:
    first_line = json.loads(f.readline())
    print(f"Question: {first_line['question']}")
    print(f"Label: {first_line['require_clarification']}|{first_line['category']}|{first_line['subclass']}")
    print(f"Text preview: {first_line['text'][:200]}...")
